In [1]:
import numpy as np
import pandas as pd
from collections import Counter
from hint_search import read_json
from hint_search import calculate_word_value
from hint_search import is_correct_word
from hint_search import is_available_word

# Считываем словарь

In [2]:
rus_words_data = pd.read_csv('word_rus.txt', 
                             header=None,
                             names=['word'])
rus_words_data

,word
0,абажур
1,абаз
2,абазин
3,абазинка
4,абака
...,...
34005,ящурка
34006,ёкание
34007,ёлка
34008,ёмкость


# Добавляем колонку с длиной слова

In [3]:
rus_words_data['length'] = rus_words_data.word.apply(lambda x: len(x))
rus_words_data

,word,length
0,абажур,6
1,абаз,4
2,абазин,6
3,абазинка,8
4,абака,5
...,...,...
34005,ящурка,6
34006,ёкание,6
34007,ёлка,4
34008,ёмкость,7


# Убираем слова длинее 15 букв

In [4]:
rus_words_data = rus_words_data.query('length <= 15')
rus_words_data

,word,length
0,абажур,6
1,абаз,4
2,абазин,6
3,абазинка,8
4,абака,5
...,...,...
34005,ящурка,6
34006,ёкание,6
34007,ёлка,4
34008,ёмкость,7


# Убираем слова с неожиданными символами и содержащие больше букв, чем есть в игре

In [5]:
rus_words_data['is_correct'] = rus_words_data.word.apply(lambda x: is_correct_word(x, 'letters_values.json'))
rus_words_data = rus_words_data.query('is_correct == True')
rus_words_data.drop('is_correct', axis=1) # больше не нужна

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,word,length
0,абажур,6
1,абаз,4
2,абазин,6
3,абазинка,8
4,абака,5
...,...,...
34001,ящер,4
34002,ящерица,7
34003,ящик,4
34004,ящур,4


In [6]:
rus_words_data['value'] = rus_words_data.word.apply(lambda x: calculate_word_value(x, 'letters_values.json'))
rus_words_data

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,word,length,is_correct,value
0,абажур,6,True,15
1,абаз,4,True,10
2,абазин,6,True,12
3,абазинка,8,True,15
4,абака,5,True,8
...,...,...,...,...
34001,ящер,4,True,16
34002,ящерица,7,True,28
34003,ящик,4,True,16
34004,ящур,4,True,18


# Отранжируем слова по цене (без учета бонусных полей на доске).

In [7]:
rus_words_data.sort_values(by='value', ascending=False)

,word,length,is_correct,value
32910,шлифовальщица,13,True,56
32862,шифровальщица,13,True,56
33274,экзаменующаяся,14,True,54
32918,шлихтовальщица,14,True,53
2370,брошюровщица,12,True,53
...,...,...,...,...
6926,ер,2,True,3
17137,ом,2,True,3
885,ар,2,True,3
299,ад,2,True,3


# Найдем все слова, которые можем составить с заданным набором букв.

In [10]:
rus_words_data['available'] = rus_words_data.word.apply(lambda x: is_available_word(Counter('абвгдеё'), x))
rus_words_data.query('available == True')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,word,length,is_correct,value,available
299,ад,2,True,3,True
1471,бег,3,True,7,True
2786,веда,4,True,6,True
6868,еда,3,True,4,True


# Из букв 'абвгдеё' можно составить вышеперчисленные слова.